In [7]:
from usefulclasses import Node, LinkedList, Stack, Queue, heapsort
import json

In [8]:
with open('books.json') as file:
    books = json.load(file)

In [9]:
class Book:
    def __init__(self, author: str, desc: str, genre: list, isbn: str, pages: int, rating: int, title: str, totalratings: int):
        self.author = author                    # Initialisation of the author of the book
        self.desc = desc                        # Short description of the content of the book
        self.genre = genre                      # List that provides the many genres that the book lies within 
        self.isbn = isbn                        # ID of the book, used to ease the access to the book objects
        self.pages = pages                      # Number of pages in each book, other mean of recommendation
        self.rating = rating                    # A ratiing on 5, with 5 being the most recommendable and 0 the least
        self.title = title                      # Initialisation of the title of the book
        self.totalratings = totalratings        # The total number of given ratings to the book, popularity credential
        self.rate = self.rating / self.totalratings if self.totalratings != 0 else 0 # rate for sorting
        
    def __str__(self):
        return self.title + " by " + self.author + "\trate: " + str(self.rate)
    
    

In [10]:
books_list = []
for book in books:
    author, desc, genre, isbn, pages, rating, title, totalratings = list(book.values())
    obj = Book(author, desc, genre, isbn, pages, rating, title, totalratings)
    books_list.append(obj)

# sort books in ascending order
heapsort(books_list)

In [12]:
class user:
    def __init__(self, id: int, name: str, fav_genres: LinkedList, library: Queue, wishlist: Queue):
        self.id =id
        self.name = name
        self.fav_genres = fav_genres
        self.library = library
        self.wishlist = wishlist
        
    def read_book(self):
        pass
    def rate_book(self):
        pass
    def add_to_wishlist(self):
        pass

    def search_author(self, author: str) -> LinkedList: 
        books = LinkedList()
        for book in books_list:
            if author in book.author:
                books.prepend(book)
        return books

    def search_book(self, title: str) -> LinkedList:
        books = LinkedList()
        for book in books_list:
            if title in book.title:
                books.prepend(book)
        return books
    
    def search_book_isbn(self, isbn: str) -> LinkedList:
        books = LinkedList()
        for book in books_list:
            if isbn in book.isbn:
                books.prepend(book)
        return books

    def search_genre(self, genre: str) -> LinkedList:
        books = LinkedList()
        for book in books_list:
            if genre in book.genre:
                books.prepend(book)
        return books

    def search_keywords(self):
        pass
    
    def most_checked_genre(self) -> str:
        lib = self.library
        
        
    def most_checked_author(self) -> str:
        pass
    
    def recommend_books_genre(self) -> dict:
        books = dict()
        genre = self.fav_genres
        while genre is not None:
            count = 3
            books[genre.val] = LinkedList()
            for book in books_list:
                if genre.val in book.genre and count != 0:
                    books[genre.val].append(book) 
                    count -= 1
            genre = genre.next
        return books
